In [ ]:
# import web driver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC


import requests
import time
import random
import re
from seleniumrequests import Chrome
import getpass
import os
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup

In [86]:
# specifies the path to the chromedriver.exe
#driver = webdriver.Chrome('chromedriver.exe')

# # For PC
# driver = Chrome()

# For Mac
git_folder_location = os.path.abspath(os.path.dirname('metis_proj_2_luther'))
full_path_to_chromedriver = os.path.join(git_folder_location, "chromedriver")
driver = Chrome(executable_path = full_path_to_chromedriver)

# driver.get method() will navigate to a page given by the URL address
driver.get('https://www.linkedin.com/login?trk=guest_homepage-basic_nav-header-signin')

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Enter ID
username = driver.find_element_by_name("session_key")
username_input = getpass.getpass("LinkedIn Username: ")
username.send_keys(username_input)

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Enter Password
password = driver.find_element_by_name("session_password")
password_input = getpass.getpass("LinkedIn Password: ")
password.send_keys(password_input)

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Click Submit Button
submit = driver.find_element_by_xpath('//*[@type="submit"]')
submit.click()

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Navigate to Metis Alumni page
driver.get('https://www.linkedin.com/school/metis/people/?facetGeoRegion=us%3A0')

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Load Entire Page by Scrolling to the Bottom
SCROLL_PAUSE_TIME = 2# Scroll to Very Bottom to Load All
last_height = driver.execute_script("return document.body.scrollHeight")# Get scroll height
while True:    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll down to bottom
    time.sleep(SCROLL_PAUSE_TIME) # Wait to load page    
    new_height = driver.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
    if new_height == last_height:
        break
    last_height = new_height

LinkedIn Username: ········
LinkedIn Password: ········


In [33]:
# Load source code of fully loaded page
page_source = driver.page_source

# Obtain LinkedIn url of All Alumni whose profiles are public
linkedin_url_list_dirty = []
for m in re.finditer('people_profile_card_name_link',page_source):
    linkedin_url_list_dirty.append(page_source[m.end()+8:m.end()+60])
linkedin_url_list = [url.split()[0][:-1] for url in linkedin_url_list_dirty]

In [34]:
# Total parsed LinkedIn addresses of Metis Alumni
total_num_alumni = page_source.count('artdeco-entity-lockup--stacked-center')
total_num_alumni_public = page_source.count('people_profile_card_name_link')
print("There are {} Metis Alumni, but {} of their profiles are private.  Hence, only {}'s linkedin addresses were parsed.".format(total_num_alumni,total_num_alumni-total_num_alumni_public,total_num_alumni_public))

# # Example of parsed LinkedIn addresses
# print("Example of parsed LinkedIn addresses :")
# linkedin_url_list[:5]

There are 151 Metis Alumni, but 6 of their profiles are private.  Hence, only 145's linkedin addresses were parsed.


In [49]:
# Test list
#linkedin_url_list_temp = ["/in/matthew-eungoo-lee/"]
linkedin_url_list_temp = [linkedin_url_list[13]]

# Function to Scrape data off of LinkedIn profile
def scrape_linkedin_profile(url):
    # Keep using same tab for each url
    driver.get(url)
    
#     # Create new tap for each url. This block doesn't work. It's not really needed for project though.
#     print(url)
#     script_text = "'''window.open('"+url+ "','_blank');'''"
#     driver.execute_script(script_text)

    # Sleep for Random Seconds Between 1 and 3
    time.sleep(random.randint(10,30)/10)
    
    
# Execute scrape_linkedin_profile function for each profile
for url in linkedin_url_list_temp: # Replace 'linkedin_url_list_temp' with 'linkedin_url_list' after testing
    scrape_linkedin_profile('https://www.linkedin.com'+url)
    

In [133]:
# # Opening in new tabs. Below two codes work.
# driver.execute_script('''window.open("https://www.linkedin.com/in/matthew-eungoo-lee/",'_blank');''')
# driver.execute_script('''window.open("https://www.google.com","_blank");''')

**Tip from Joe**

Here's a short summary of how you would likely develop and structure scraping code:
1. Starting with one page, write parsing functions that should generalize to any page with the same format.
2. Test your code on several different pages to confirm that it works in general. Try to make your code robust by checking returns for None and using try;except clauses to accommodate missing elements like in the get_movie_value function above.
3. Figure out the total collection of webpages that you want to scrape, and collect the urls into a list (e.g. 2018 movies 1-100, 101-200, etc.). Iterating through all the urls, request and parse the page, adding the data to a list of dicts.
4. Incrementally convert the list of dicts into a dataframe and save it to disk with pd.to_csv().
Note that this workflow leaves out some details you may need like intentional pausing, but we'll get to these soon :) 
Also, this is a suggestion, not a prescription. You should think of project luther as a first foray into setting up a data acquisition pipeline, and it takes some intelligent and creative design to get it right. Our best advice is to start scraping early and see where things break quickly so that you can fix it! 

[LinkedIn Scrape Data](https://www.linkedin.com/pulse/how-easy-scraping-data-from-linkedin-profiles-david-craven/)  
[Metis Alumni LinkedIn](https://www.linkedin.com/school/metis/people/?facetGeoRegion=us%3A0)



Conclusion: Beautiful Soup is powerful but it has many limitations. If a page needs interactions (like entering password) or if a page is not static, but dynamically generated, we can't use Soup. We will explore other tools for that.

One such tool for tomorrow: *Selenium*

In [ ]:
# specifies the path to the chromedriver.exe
#driver = webdriver.Chrome('chromedriver.exe')

# # For PC
# driver = Chrome()

# For Mac
git_folder_location = os.path.abspath(os.path.dirname('metis_proj_2_luther'))
full_path_to_chromedriver = os.path.join(git_folder_location, "chromedriver")
driver = Chrome(executable_path = full_path_to_chromedriver)

# driver.get method() will navigate to a page given by the URL address
driver.get('https://www.linkedin.com/login?trk=guest_homepage-basic_nav-header-signin')

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Enter ID
username = driver.find_element_by_name("session_key")
username_input = getpass.getpass("LinkedIn Username: ")
username.send_keys(username_input)

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Enter Password
password = driver.find_element_by_name("session_password")
password_input = getpass.getpass("LinkedIn Password: ")
password.send_keys(password_input)

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Click Submit Button
submit = driver.find_element_by_xpath('//*[@type="submit"]')
submit.click()

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Navigate to Metis Alumni page
driver.get('https://www.linkedin.com/school/metis/people/?facetGeoRegion=us%3A0')

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Load Entire Page by Scrolling to the Bottom
SCROLL_PAUSE_TIME = 2# Scroll to Very Bottom to Load All
last_height = driver.execute_script("return document.body.scrollHeight")# Get scroll height
while True:    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll down to bottom
    time.sleep(SCROLL_PAUSE_TIME) # Wait to load page    
    new_height = driver.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
    if new_height == last_height:
        break
    last_height = new_height

# Load source code of fully loaded page
page_source = driver.page_source

# Obtain LinkedIn url of All Alumni whose profiles are public
linkedin_url_list_dirty = []
for m in re.finditer('people_profile_card_name_link',page_source):
    linkedin_url_list_dirty.append(page_source[m.end()+8:m.end()+60])
linkedin_url_list = [url.split()[0][:-1] for url in linkedin_url_list_dirty]

# Total parsed LinkedIn addresses of Metis Alumni
total_num_alumni = page_source.count('artdeco-entity-lockup--stacked-center')
total_num_alumni_public = page_source.count('people_profile_card_name_link')
print("There are {} Metis Alumni, but {} of their profiles are private.  Hence, only {}'s linkedin addresses were parsed.".format(total_num_alumni,total_num_alumni-total_num_alumni_public,total_num_alumni_public))

# # Example of parsed LinkedIn addresses
# print("Example of parsed LinkedIn addresses :")
# linkedin_url_list[:5]

# Test list
#linkedin_url_list_temp = ["/in/matthew-eungoo-lee/"]
linkedin_url_list_temp = [linkedin_url_list[13]]

# Function to Scrape data off of LinkedIn profile
def scrape_linkedin_profile(url):
    # Keep using same tab for each url
    driver.get(url)
    
#     # Create new tap for each url. This block doesn't work. It's not really needed for project though.
#     print(url)
#     script_text = "'''window.open('"+url+ "','_blank');'''"
#     driver.execute_script(script_text)

    # Sleep for Random Seconds Between 1 and 3
    time.sleep(random.randint(10,30)/10)
    
    
# Execute scrape_linkedin_profile function for each profile
for url in linkedin_url_list_temp: # Replace 'linkedin_url_list_temp' with 'linkedin_url_list' after testing
    scrape_linkedin_profile('https://www.linkedin.com'+url)
    

# # Opening in new tabs. Below two codes work.
# driver.execute_script('''window.open("https://www.linkedin.com/in/matthew-eungoo-lee/",'_blank');''')
# driver.execute_script('''window.open("https://www.google.com","_blank");''')

**Tip from Joe**

Here's a short summary of how you would likely develop and structure scraping code:
1. Starting with one page, write parsing functions that should generalize to any page with the same format.
2. Test your code on several different pages to confirm that it works in general. Try to make your code robust by checking returns for None and using try;except clauses to accommodate missing elements like in the get_movie_value function above.
3. Figure out the total collection of webpages that you want to scrape, and collect the urls into a list (e.g. 2018 movies 1-100, 101-200, etc.). Iterating through all the urls, request and parse the page, adding the data to a list of dicts.
4. Incrementally convert the list of dicts into a dataframe and save it to disk with pd.to_csv().
Note that this workflow leaves out some details you may need like intentional pausing, but we'll get to these soon :) 
Also, this is a suggestion, not a prescription. You should think of project luther as a first foray into setting up a data acquisition pipeline, and it takes some intelligent and creative design to get it right. Our best advice is to start scraping early and see where things break quickly so that you can fix it! 

[LinkedIn Scrape Data](https://www.linkedin.com/pulse/how-easy-scraping-data-from-linkedin-profiles-david-craven/)  
[Metis Alumni LinkedIn](https://www.linkedin.com/school/metis/people/?facetGeoRegion=us%3A0)



Conclusion: Beautiful Soup is powerful but it has many limitations. If a page needs interactions (like entering password) or if a page is not static, but dynamically generated, we can't use Soup. We will explore other tools for that.

One such tool for tomorrow: *Selenium*